<a href="https://colab.research.google.com/github/kikiru328/Bone_Detection/blob/main/Final_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

공용폴더에 폴더(yolov5), 파일(model.pt) 업로드해뒀습니다.

In [ ]:
def input_info(NO, gender):
  global img_path, model_path, tjnet_path

  # Gender condition
  if gender == 1:
    ge = 'M'
    sex = 'male'
  elif gender == 0:
    ge   = 'F'
    sex = 'female'

  img_path =f'/content/drive/MyDrive/2차 프로젝트 원본 데이터/preprocessing_done/{sex}/{NO}_{ge}.jpg'
  model_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/yolov5'
  tjnet_path = '/content/drive/MyDrive/2차 프로젝트 원본 데이터/TJM/tjnet24.h5'

  return [NO, gender, img_path, model_path, tjnet_path]

def predict_age(NO, gender):
  import pandas as pd
  df = pd.read_pickle('/content/drive/MyDrive/2차 프로젝트 원본 데이터/BA_all_.pkl')
  # df.columns
  ba_mean = df.BONE.mean()
  ba_std = df.BONE.std()
  return ba_mean, ba_std, df
  

def predict_zscore(gender, img_path, model_path,tjnet_path, show_crop, result_out):
  yolo_crop_img(gender, img_path, model_path, show_crop=show_crop,result_out = result_out)
  import numpy as np
  import tensorflow.keras as tf
  model = tf.models.load_model(tjnet_path, compile=False)
  # grobal X : X_ray 이미지의 yolo crop image 값
  y_predict = model.predict(X)
  global pred
  pred = y_predict[0][0]

def yolo_crop_img(gender, img_path, model_path, result_out=False, show_crop=False):
  import os
  import numpy as np
  os.chdir(model_path)
  import torch
  model = torch.load('./model.pt', map_location='cpu')
  result = model(img_path)
  global crops
  crops = result.crop(save=False)
  global X
  X = out_crop_img(crops, gender)
  global img
  img = np.squeeze(result.render())
  if show_crop: show_img(result)
  if result_out: return result

def show_img(result):
  import matplotlib.pyplot as plt
  import numpy as np
  %matplotlib inline
  plt.figure(figsize=(16,12))
  plt.imshow(np.squeeze(result.render()))
  plt.show()

def out_crop_img(crop, gender):
  import re
  import cv2
  import numpy as np

  gender = np.array(gender).reshape(1,1)

  for i in range(7):
    carpal = re.compile('CARPAL.')
    ip = re.compile('IP.')
    lmcp = re.compile('LMCP.')
    lpip = re.compile('LPIP.')
    mmcp = re.compile('MMCP.')
    mpip = re.compile('MPIP.')
    tmcp = re.compile('TMCP.')

    if carpal.search(crop[i]['label']):
      CARPAL_img = crop[i]['im']
      CARPAL_img = cv2.resize(CARPAL_img, (224,224),cv2.INTER_AREA)
      CARPAL_img = np.expand_dims(CARPAL_img, axis=0)

    if ip.search(crop[i]['label']):
      IP_img = crop[i]['im']
      IP_img = cv2.resize(IP_img, (75,75),cv2.INTER_AREA)
      IP_img = np.expand_dims(IP_img, axis=0)
      
    if lmcp.search(crop[i]['label']):
      LMCP_img = crop[i]['im']
      LMCP_img = cv2.resize(LMCP_img, (75,75),cv2.INTER_AREA)
      LMCP_img = np.expand_dims(LMCP_img, axis=0)

    if lpip.search(crop[i]['label']):
      LPIP_img = crop[i]['im']
      LPIP_img = cv2.resize(LPIP_img, (75,75),cv2.INTER_AREA)
      LPIP_img = np.expand_dims(LPIP_img, axis=0)
    
    if mmcp.search(crop[i]['label']):
      MMCP_img = crop[i]['im']
      MMCP_img = cv2.resize(MMCP_img, (75,75),cv2.INTER_AREA)
      MMCP_img = np.expand_dims(MMCP_img, axis=0)
    
    if mpip.search(crop[i]['label']):
      MPIP_img = crop[i]['im']
      MPIP_img = cv2.resize(MPIP_img, (75,75),cv2.INTER_AREA)
      MPIP_img = np.expand_dims(MPIP_img, axis=0)

    if tmcp.search(crop[i]['label']):
      TMCP_img = crop[i]['im']
      TMCP_img = cv2.resize(TMCP_img, (75,75),cv2.INTER_AREA)
      TMCP_img = np.expand_dims(TMCP_img, axis=0)

    else : continue

  
  return [CARPAL_img, LMCP_img, MMCP_img,TMCP_img, LPIP_img, MPIP_img, IP_img, gender]

In [ ]:
def final(NO,gender,result_out=True,show_crop=True,):
  NO, gender, img_path, model_path, tjnet_path = input_info(NO,gender)
  ba_mean, ba_std, df = predict_age(NO,gender)
  predict_zscore(gender, img_path, model_path,tjnet_path, show_crop, result_out)
  prediction_BA = (pred * ba_std + ba_mean)/12
  cond1 = df['GENDER'] == gender
  cond2 = df['NO'] == NO
  Actual_value = df[cond1&cond2]['BA_TOTAL'].values[0]
  Diff = abs( Actual_value - prediction_BA )*12
  print('Actual > {0} Year'.format( round(Actual_value,2)) )
  print('prediction_BA > {0} Year'.format( round(prediction_BA,2)))
  print('Different > {0} Month'.format(round(Diff,2)))
  return Diff
  

final(1,0,True,True)